In [1]:
import pandas as pd, numpy as np
import vivarium_output_loader as vol
import lsff_output_processing as lop
import lsff_summarizer
from lsff_summarizer import LSFFOutputSummarizer

!whoami
!date

ndbs
Fri Jun  5 14:31:17 PDT 2020


In [5]:
%load_ext autoreload
%autoreload 2

# Check whether we have the correct observers in model 7

## 1. Define directories

In [2]:
base_directory = '/share/costeffectiveness/results/vivarium_conic_lsff/00_bugfix'
india_v7_directory = base_directory + '/v7.0_prelim_verify_result_categories/india'
locations_paths = {'India': india_v7_directory}

## 2. Load raw output

In [6]:
raw_output = vol.load_output_by_location(locations_paths)
raw_output.head()

total_population_untracked  \
input_draw_number random_seed                               
602.0             6.0                              9913.0   
                  53.0                             9921.0   
946.0             53.0                             9909.0   
829.0             6.0                              9916.0   
946.0             5.0                              9917.0   

                               total_population_tracked  total_population  \
input_draw_number random_seed                                               
602.0             6.0                            9533.0           19446.0   
                  53.0                           9794.0           19715.0   
946.0             53.0                           9718.0           19627.0   
829.0             6.0                            9442.0           19358.0   
946.0             5.0                            9603.0           19520.0   

                               diarrheal_diseases_prevalent_cases_at_sim_end  \
input_draw_number random_seed                                                  
602.0             6.0                                                  603.0   
                  53.0                                                 643.0   
946.0             53.0                                                 676.0   
829.0             6.0                                                  558.0   
946.0             5.0                                                  667.0   

                               susceptible_to_diarrheal_diseases_event_count  \
input_draw_number random_seed                                                  
602.0             6.0                                                19361.0   
                  53.0                                               20324.0   
946.0             53.0                                               20558.0   
829.0             6.0                                                17706.0   
946.0             5.0                                                20744.0   

                               diarrheal_diseases_event_count  \
input_draw_number random_seed                                   
602.0             6.0                                 20159.0   
                  53.0                                21125.0   
946.0             53.0                                21399.0   
829.0             6.0                                 18414.0   
946.0             5.0                                 21583.0   

                               measles_prevalent_cases_at_sim_end  \
input_draw_number random_seed                                       
602.0             6.0                                      1731.0   
                  53.0                                     1879.0   
946.0             53.0                                      823.0   
829.0             6.0                                      2262.0   
946.0             5.0                                       857.0   

                               susceptible_to_measles_event_count  \
input_draw_number random_seed                                       
602.0             6.0                                         0.0   
                  53.0                                        0.0   
946.0             53.0                                        0.0   
829.0             6.0                                         0.0   
946.0             5.0                                         0.0   

                               measles_event_count  \
input_draw_number random_seed                        
602.0             6.0                       1774.0   
                  53.0                      1920.0   
946.0             53.0                       850.0   
829.0             6.0                       2318.0   
946.0             5.0                        891.0   

                               recovered_from_measles_event_count  ...  \
input_draw_number random_seed                                   

## 3. Create an output summarizer and print column report

Initial categorized output had the following missing and repeated columns:

```
Missing (12 data column(s) not captured in a category):
 {'hemoglobin_variance_at_age_1_status_uncovered_responsive_non-responsive', 'hemoglobin_variance_at_age_1_status_covered_responsive_non-responsive', 'hemoglobin_variance_at_age_3_status_covered_responsive_responsive', 'hemoglobin_variance_at_age_2_status_uncovered_responsive_non-responsive', 'hemoglobin_variance_at_age_1_status_covered_responsive_responsive', 'hemoglobin_variance_at_age_3_status_uncovered_responsive_non-responsive', 'hemoglobin_variance_at_age_2_status_covered_responsive_non-responsive', 'hemoglobin_variance_at_age_1_status_uncovered_responsive_responsive', 'hemoglobin_variance_at_age_3_status_uncovered_responsive_responsive', 'hemoglobin_variance_at_age_2_status_uncovered_responsive_responsive', 'hemoglobin_variance_at_age_2_status_covered_responsive_responsive', 'hemoglobin_variance_at_age_3_status_covered_responsive_non-responsive'}

Repeated (4 data column(s) appearing in more than one category):
 {'hemoglobin_mean_at_age_0.5_status_covered_responsive_non-responsive': 2, 'hemoglobin_mean_at_age_0.5_status_covered_responsive_responsive': 2, 'hemoglobin_mean_at_age_0.5_status_uncovered_responsive_responsive': 2, 'hemoglobin_mean_at_age_0.5_status_uncovered_responsive_non-responsive': 2}
```

The repeated columns ended up in the `'intervention'` category as well as the `'distribution_mean'` category:

```
In  [8]: output.columns('intervention')

Out [8]: ['hemoglobin_mean_at_age_0.5_status_covered_responsive_non-responsive',
          'hemoglobin_variance_at_age_0.5_status_covered_responsive_non-responsive',
          'hemoglobin_mean_at_age_0.5_status_covered_responsive_responsive',
          'hemoglobin_variance_at_age_0.5_status_covered_responsive_responsive',
          'hemoglobin_mean_at_age_0.5_status_uncovered_responsive_responsive',
          'hemoglobin_variance_at_age_0.5_status_uncovered_responsive_responsive',
          'hemoglobin_mean_at_age_0.5_status_uncovered_responsive_non-responsive',
          'hemoglobin_variance_at_age_0.5_status_uncovered_responsive_non-responsive',
          'fortification_intervention.scenario']

```

* Variance columns were missing because this is a new type of output (previously we had standard deviation), so I added a new category.

* Some of the mean and variance columns ended up in the `'intervention'` category because that caught any column with a `.`; I changed this to exclude a decimal point preceded by a digit.

In [10]:
output = LSFFOutputSummarizer(raw_output)
output.print_column_report()

Number of data columns in output: 15450
Total number of columns captured in categories: 15450

Number of columns in each category:
 {'input_draw': 1, 'random_seed': 1, 'location': 1, 'intervention': 1, 'run_time': 0, 'diseases_at_end': 4, 'transition_count': 3465, 'population': 5, 'person_time': 432, 'state_person_time': 4320, 'treated_days': 0, 'mortality': 2160, 'total_daly': 2, 'yld': 2592, 'yll': 2160, 'categorical_risk': 0, 'birth_prevalence': 108, 'live_births': 108, 'proportion': 2, 'distribution_mean': 44, 'distribution_variance': 16, 'distribution_std_dev': 28, 'prevalence': 0} 

Missing (0 data column(s) not captured in a category):
 set()

Repeated (0 data column(s) appearing in more than one category):
 {}

Empty categories (4 categories with no matching data columns):
 ['run_time', 'treated_days', 'categorical_risk', 'prevalence']


In [11]:
output.columns('intervention')

['fortification_intervention.scenario']

## 4. Check scenarios

In [12]:
output.subdata('intervention').head()

fortification_intervention.scenario
input_draw_number random_seed                                    
602.0             6.0           folic_acid_fortification_scale_up
                  53.0                iron_fortification_scale_up
946.0             53.0                                   baseline
829.0             6.0                 iron_fortification_scale_up
946.0             5.0                 iron_fortification_scale_up

In [14]:
output.data['fortification_intervention.scenario'].unique()

array(['folic_acid_fortification_scale_up', 'iron_fortification_scale_up',
       'baseline', 'vitamin_a_fortification_scale_up'], dtype=object)

## 5. Check mean, gestational age, and proportion columns

In [15]:
output.columns('distribution_mean')

['hemoglobin_mean_at_age_0.5_status_covered_responsive_non-responsive',
 'hemoglobin_mean_at_age_1_status_covered_responsive_non-responsive',
 'hemoglobin_mean_at_age_2_status_covered_responsive_non-responsive',
 'hemoglobin_mean_at_age_3_status_covered_responsive_non-responsive',
 'birth_weight_mean_in_2020_among_female_iron_fortification_group_covered',
 'birth_weight_mean_in_2020_among_female_iron_fortification_group_uncovered',
 'birth_weight_mean_in_2020_among_female_iron_fortification_group_unknown',
 'birth_weight_mean_in_2020_among_male_iron_fortification_group_covered',
 'birth_weight_mean_in_2020_among_male_iron_fortification_group_uncovered',
 'birth_weight_mean_in_2020_among_male_iron_fortification_group_unknown',
 'birth_weight_mean_in_2021_among_female_iron_fortification_group_covered',
 'birth_weight_mean_in_2021_among_female_iron_fortification_group_uncovered',
 'birth_weight_mean_in_2021_among_male_iron_fortification_group_covered',
 'birth_weight_mean_in_2021_among_ma

In [17]:
raw_output.filter(regex = 'gestational').head()

gestational_age_mean  gestational_age_sd  \
input_draw_number random_seed                                             
602.0             6.0                     38.731413            2.783300   
                  53.0                    38.770020            2.138627   
946.0             53.0                    38.911646            1.695726   
829.0             6.0                     38.660235            2.414330   
946.0             5.0                     39.000252            2.052960   

                               gestational_age_proportion_below_37w  
input_draw_number random_seed                                        
602.0             6.0                                      0.142857  
                  53.0                                     0.142857  
946.0             53.0                                     0.063380  
829.0             6.0                                      0.202381  
946.0             5.0                                      0.121019

In [20]:
raw_output.filter(regex=r'^birth_weight_mean$').head()

birth_weight_mean
input_draw_number random_seed                   
602.0             6.0                2906.271910
                  53.0               3036.581773
946.0             53.0               2981.884636
829.0             6.0                3122.759134
946.0             5.0                3045.953001

In [21]:
output.columns('proportion')

['birth_weight_proportion_below_2500g', 'gestational_age_proportion_below_37w']